<a href="https://colab.research.google.com/github/adalbertogc1/UKGridBalancingCostsVisualisation/blob/main/Carbon_visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to grid electricy costs visualiser 

This collab notebook displays the UK-level grid balancing costs from the ([National Grid](https://data.nationalgrideso.com/) alongside regional information from the [Carbon Intensity API](https://www.carbonintensity.org.uk/) such as the type of energy by source or the carbon intesity for the **next 48 hours.**

Information is presented at half-hourly level.

Created by: Adalberto Guerra Cabrera (guerraca@icloud.com)


### To start, run the initial cell to import dependencies and define some variables

In [ ]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import requests 
import json
import pytz
import plotly.express as px

utc=pytz.UTC

region_ids ={"North Scotland":1,"South Scotland":2,"North West England":3,"North East England":4,"Yorkshire":5,
             "North Wales & Merseyside":6,"South Wales":7,"West Midlands":8,"East Midlands":9,"East England":10,
             "South West England":11,"South England":12,"London":13,"South East England":14,"England":15,"Scotland":16,"Wales":17}

show_options = ['source', 'carbon']

energy_sources = ("biomass","coal","imports","gas","nuclear","other","hydro","solar","wind")


### Select the UK region and type of context information (energy source or carbon intensity)

In [ ]:

# select region
region_id= region_ids["Wales"]

# select what you want to see ([0] energy source or [1]carbon intensity)
show = show_options[0]

# if energy source if selected, select specific energy source 
# ([0] "biomass", [1] "coal", [2] "imports", [3] "gas",[4] "nuclear",[5] "other",[6] "hydro",[7] "solar",[8] "wind")
energy_source = energy_sources[3]

### Get the information from the data sources and visualise them in an interactive bar plot

In [ ]:

def function(x):
    x=x-1
    if x <= 24:
        foo = x % 2
        if foo != 0:
            temp = "0"+str(int(x/2))+":30"
        else:
            temp = "0"+str(int(x/2))+":00"
    else:
        if x%2 ==1:
            temp = str(int(x/2))+":30"
        else:
            temp = str(int(x/2))+":00"
    return temp
n_days: int = 1
start_datetime = utc.localize(datetime.now())
end_datetime = start_datetime +timedelta(days=n_days)
filename =f"daily-bsuos-forecast-{end_datetime.date()}.csv"
try: 
    forecast_df = pd.read_csv(filename, parse_dates=['Datetime'])
    forecast_df.set_index('Datetime', inplace=True)
    forecast_df.index = pd.to_datetime(forecast_df.index, utc=True)
    selected_forecast_df=forecast_df.loc[start_datetime:end_datetime]
    if len(selected_forecast_df) <(48*n_days):
            raise ValueError("WARNING: No data found in the specified dates. Downloading data...")
except Exception as e:
    url = requests.get("https://data.nationalgrideso.com/balancing/balancing-services-use-of-system-bsuos-daily-forecast/datapackage.json")
    data = json.loads(url.text)
    forecast_date = data['metadata_modified'][:10]
    print(f"WARNING: {e}. Downloading data from National Grid.")
    forecast_csv_URL = f"https://data.nationalgrideso.com/backend/dataset/a8368560-87ac-4d31-85b7-625322430456/resource/586d0e6b-cc03-4312-9df3-407438715eb2/download/daily-bsuos-forecast-{start_datetime.year-1}-{str(start_datetime.year)[:-2]}.csv"
    #settled_cost_URL = "https://data.nationalgrideso.com/backend/dataset/d6a4bf54-c63f-4014-a716-49fd3878ca52/resource/26b0f410-27d4-448a-9437-45277818b838/download/current_rf_bsuos_data.csv"
    forecast_df = pd.read_csv(forecast_csv_URL,encoding = "ISO-8859-1")
    forecast_df = forecast_df[["Date", "SP","BSUoS £/MWh"]]
    forecast_df['Date'] = pd.to_datetime(forecast_df['Date'])
    forecast_df['Time'] = forecast_df['SP'].apply(function)
    forecast_df['Datetime'] = pd.to_datetime(forecast_df['Date'].apply(str)+' '+forecast_df['Time'])
    forecast_df.drop(["Date","SP","Time"], axis=1, inplace=True)
    forecast_df.set_index('Datetime', inplace=True)
    forecast_df.to_csv(filename)
    forecast_df.index = pd.to_datetime(forecast_df.index, utc=True)
    selected_forecast_df=forecast_df.loc[start_datetime:end_datetime]
    if len(selected_forecast_df) <(48*n_days):
            raise ValueError("No data found in the specified dates. Try to reduce the number of forecast days.")
    else:
        print(f"INFO: Data from National grid downloaded.")

### Carbon Intensity for Scotland
#Information obtained from <https://www.carbonintensity.org.uk/>
# https://carbon-intensity.github.io/api-definitions/#get-regional


filename = "regionId_{}_date_{}.json".format(region_id,start_datetime.date())

try:
    with open(filename, 'r') as f:
        payload = json.loads(f.read())
except Exception as e:
    print(f"INFO: Exception: {e}.\n")
    print("INFO: Downloading data from carbonintensity.org.uk ")
    
    carbon_url = f'https://api.carbonintensity.org.uk/regional/intensity/{start_datetime.isoformat()}/fw48h/regionid/{region_id}'
    headers = {'Accept': 'application/json'}
    response = requests.get(carbon_url, params={}, headers = headers) 
    payload = response.json()
    with open(filename, 'w+') as f:
        json.dump(payload, f, indent=4)
    print("INFO: Data from carbonintensity.org.uk downloaded.")
intensity= {}

short_name = payload['data']['shortname']
for data in payload['data']['data']:
    intensity[data['to']]={}
    intensity[data['to']]['intensity'] = data['intensity']['forecast']
    intensity[data['to']]['index'] = data['intensity']['index']
    for fuel in data['generationmix']:
        intensity[data['to']][fuel['fuel']]= float(fuel['perc'])

    
intensity_df = pd.DataFrame.from_dict(intensity, orient='index')
intensity_df.index = pd.to_datetime(intensity_df.index, utc=True)

df =selected_forecast_df.merge(intensity_df.loc[start_datetime:end_datetime], how='inner', left_index=True, right_index=True)
df['Datetime']= df.index

if show == 'source':
    color_data = energy_source
else:
    color_data = 'intensity'

fig = px.bar(df, x='Datetime', y='BSUoS £/MWh',
             hover_data=['BSUoS £/MWh', 'intensity'], color=color_data,
             labels={'BSUoS £/MWh':'Electricity cost (£/MWh)','gas':'% of the mix coming from gas','nuclear':'% of the mix coming from nuclear',
                    'solar':'% of the mix coming from solar', 'wind':'% of the mix coming from wind',
                    'intensity': 'Carbon intensity (gCO2/kWh)'}, height=400,
            title=f"Electricity cost for the next 48h for the {short_name} region", color_continuous_scale="Bluered")
fig.show()

INFO: Exception: [Errno 2] No such file or directory: 'regionId_17_date_2021-09-27.json'.

INFO: Downloading data from carbonintensity.org.uk 
INFO: Data from carbonintensity.org.uk downloaded.
